In [4]:
!pip install pytorch_lightning
!git clone https://github.com/black0017/MedicalZooPytorch.git
!pip install -r MedicalZooPytorch/installation/requirements.txt
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 675kB 19.5MB/s 
     |████████████████████████████████| 276kB 60.8MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 829kB 50.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=4b21acdfed7f6a1e5f1ffffa44f9c55f998fbfad5656e83086eb0bee19efe22a
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=17ab74dd559bb51ce4fee2dcc0572f27a88f32ac5556d2faa92fd77004bd8147
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uni

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 17kB/s 
     |████████████████████████████████| 12.8MB 59.4MB/s 
     |████████████████████████████████| 7.6MB 7.5MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
import sys
sys.path.append('./MedicalZooPytorch')
from lib import medzoo
import nibabel as nb
from skimage import transform

In [11]:
#Pytorch-lightning setup
class TumourSegmentation(pl.LightningModule):
  def __init__(self,model):
    super().__init__()
    self.model = model
  def forward(self,x):
    x=x.float()
    return self.model.forward(x)
  def training_step(self, batch, batch_idx):
    x, y = batch
    x=x.float()
    y=y.float()
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    return loss
  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=0.02)


In [12]:
#model initialization
unet_model =  medzoo.Unet3D.UNet3D(in_channels=4,n_classes=2,base_n_filter=8)
model = TumourSegmentation(unet_model)

In [5]:
#Dataset
class brats_dataset(torch.utils.data.Dataset):
  def __init__(self,data_folders):
    self.data_list = []
    
    #Perform necessary input data preparation in this function
    #add each input example into the data_last function
    #takes in a list of folders and processes the data contained
    for folder in data_folders:
      f_flair = nb.load(os.path.join(folder,'BraTS20_Training_001_flair.nii'),mmap=False).get_fdata()
      f_seg = nb.load(os.path.join(folder,'BraTS20_Training_001_seg.nii'),mmap=False).get_fdata()
      f_t1ce = nb.load(os.path.join(folder,'BraTS20_Training_001_t1ce.nii'),mmap=False).get_fdata()
      f_t1 =  nb.load(os.path.join(folder,'BraTS20_Training_001_t1.nii'),mmap=False).get_fdata() 
      f_t2 = nb.load(os.path.join(folder,'BraTS20_Training_001_t2.nii'),mmap=False).get_fdata()
      
      #Resizing because UNet input shape requirements
      f_t1 = transform.resize(f_t1, [320, 400, 320])
      f_t2 = transform.resize(f_t2, [320, 400, 320])
      f_t1ce = transform.resize(f_t1ce, [320, 400, 320])
      f_flair = transform.resize(f_flair, [320, 400, 320])
      f_seg = transform.resize(f_seg, [320, 400, 320])
      self.data_list.append([np.stack([f_t1, f_t1ce, f_t2, f_flair]), f_seg])
  def __len__(self):
    return len(self.data_list)
  def __getitem__(self, index):
    return self.data_list[index]

In [6]:
data_folders = ['./drive/MyDrive/sample_data/BraTS20_Training_001']
dataset = brats_dataset(data_folders)
train_dataset, val_dataset = random_split(dataset, lengths=[1,0])

In [20]:
def train_collate_fn(batch):
  pass
  #this is where we would perform data augmentation on the input batch
  

In [9]:
#Data Loader
train_dataloader = DataLoader(train_dataset)#,collate_fn=train_collate_fn)
#val_dataloader = Dataloader(val_dataset)

In [ ]:
#finding batch size and learning rate
#trainer = pl.Trainer(auto_scale_batch_size='binsearch')
#trainer.tune(model)
#trainer = pl.Trainer(auto_lr_find = True,)
#trainer.tune(model)

In [17]:
#Training
trainer = pl.Trainer(
    accumulate_grad_batches = 1,
    gpus = 1,
    max_epochs = 1,
    #check_val_every_n_epoch = 1,

)
#trainer.tune()
trainer.fit(model=model,train_dataloader=train_dataloader)#,val_dataloaders=val_dataloader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | UNet3D | 1.8 M 
---------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


RuntimeError: ignored

In [ ]:
#Run this cell then the one below to empty gpu memory, CUDA is wierd
1/0 

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(0).total_memory
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
print(t,c,a)

/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)
